In [ ]:
from sklearn import datasets
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.inspection import DecisionBoundaryDisplay
import numpy as np
from matplotlib import pyplot as plt

## Load Dataset and Train LR classifier

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.3, random_state=17)

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

In [ ]:
classifier.coef_#weights of the logistic regression model. First row is weight vector for class 0, goes on 0-2


array([[-0.35966674,  0.84698025, -2.25952395, -0.93265309],
       [ 0.57639051, -0.32527652, -0.1716181 , -0.79325645],
       [-0.21672377, -0.52170373,  2.43114205,  1.72590954]])

## Playground (use this to test your strategy first, then generalize your code later)

In [ ]:
index = 0
x = X_test[index]
pred = classifier.predict([x])[0]
pred_probs = classifier.predict_proba([x])
print(pred, pred_probs)
#We have three different probabilities because we have three different possible flower types
#We want to choose the class closest to the one we begin from to have the easiest time crossing the decision boundary
#Say we want to change to class index 1, we need to find the features that are the most important for class 1
#so we can have the easiest time changing
#For that, we look at our earlier calculated co-efficients, only look at || Absolute value



0 [[9.85133396e-01 1.48665264e-02 7.78744897e-08]]


In [ ]:
step = 0.05
total_steps = 500
target_class = 1
feature_index = 0
sign = 1
#Input the information from the above chosen co-efficient to this

In [ ]:
x_adv = np.copy(x)
for i in range(total_steps):
    x_adv[feature_index] = x_adv[feature_index] + sign*step
    new_pred = classifier.predict([x_adv])[0]
    if (new_pred == target_class):
        print("FOUND IT")
        print("STEPS =", i+1)
        print("PREVIOUS FEATURE:", x)
        print("PREVIOUS PREDICTION", pred)
        print("NEW FEATURE:", x_adv)
        print("NEW PREDICTION", new_pred)
        break

        #Make sure that when checking the weights, the relative differences between the weights are taken into account
        #We can't change a class if the signs are the same and the current class and target class feature push
        #the model to the same conclusion. It's either inefficient, or impossible to change classes that way


FOUND IT
STEPS = 90
PREVIOUS FEATURE: [5.4 3.9 1.3 0.4]
PREVIOUS PREDICTION 0
NEW FEATURE: [9.9 3.9 1.3 0.4]
NEW PREDICTION 1


## Perturbation Strategies (Generalize)

### 1.1. Random Perturbations on All Features

In [ ]:
def strategy11(x):
    ## TODO
    raise NotImplementedError #remove this once done

### 1.2. Random Perturbations on the Most Important Feature

In [ ]:
def strategy12(x):
    ## TODO
    raise NotImplementedError #remove this once done

### 2.1 Multiple-Step Perturbations to the Closest Class on the Most Important Feature (Naive Version)

Choose the most important feature to perturb as the one that will increase the score for the target class the most

In [ ]:
def strategy21(x, step=0.05, total_steps=500, debug=False):
    pred_probs = classifier.predict_proba([x])
    pred = classifier.predict([x])[0]

    ## argsort, then reverse the list, then get the 2nd element
    target_class = np.argsort(pred_probs)[0][::-1][1]

    ## get the index of the most influential feature to the target class
    feature_index = np.argmax(np.abs(classifier.coef_[target_class]))

    ## get the sign
    sign = np.sign(classifier.coef_[target_class][feature_index])

    if debug:
        print("Target Class", target_class)
        print("Feature Index", feature_index)
        print("Step", sign*step)

    x_adv = np.copy(x)
    for i in range(total_steps):
        x_adv[feature_index] = x_adv[feature_index] + sign*step
        new_pred = classifier.predict([x_adv])[0]
        if (new_pred == target_class):
            if debug:
                print("FOUND IT")
                print("STEPS =", i+1)
                print("PREVIOUS FEATURE:", x)
                print("PREVIOUS PREDICTION", pred)
                print("NEW FEATURE:", x_adv)
                print("NEW PREDICTION", new_pred)
            return i+1, x, pred, x_adv, new_pred

    return None

In [ ]:
strategy21(X_test[1], debug=True)

### 2.2 Multiple-Step Perturbations to the Closest Class on the Most Important Feature (Pro Version)

Choose the most important feature to perturb as the one that will increase the score for the target class the most and AT THE SAME TIME decrease the score for the current class the most

In [ ]:
def strategy22(x, step=0.05, total_steps=500, debug=False):
    pred_probs = classifier.predict_proba([x])
    pred = classifier.predict([x])[0]

    ## argsort, then reverse the list, then get the 2nd element
    target_class = np.argsort(pred_probs)[0][::-1][1]

    ## get the index of the most influential feature to the target class
    for x in classifier.coef_:
      if np.sign(classifier.coef_[target_class]) != np.sign(classifier.coef_[0]):
        feature_index = np.argmax(np.abs(classifier.coef_[target_class]))

    ## get the sign
    sign = np.sign(classifier.coef_[target_class][feature_index])

    if debug:
        print("Target Class", target_class)
        print("Feature Index", feature_index)
        print("Step", sign*step)

    x_adv = np.copy(x)
    for i in range(total_steps):
        x_adv[feature_index] = x_adv[feature_index] + sign*step
        new_pred = classifier.predict([x_adv])[0]
        if (new_pred == target_class):
            if debug:
                print("FOUND IT")
                print("STEPS =", i+1)
                print("PREVIOUS FEATURE:", x)
                print("PREVIOUS PREDICTION", pred)
                print("NEW FEATURE:", x_adv)
                print("NEW PREDICTION", new_pred)
            return i+1, x, pred, x_adv, new_pred

    return None

In [ ]:
strategy22(X_test[1], debug=True)

### 3.1. Multiple-Step Perturbations to the Closest Example of A Target Class

In [ ]:
def strategy31(x, step=0.05, debug=False):
    # TODO


## Evaluation

In [ ]:
from scipy.spatial import distance
dist_func = distance.euclidean

In [ ]:
def evaluate(adv_func, dist_func, test_set):
    score = 0
    dists = 0
    for x in test_set:
        result = adv_func(x, step=0.05, total_steps=500)
        if result:
            step, x, pred, x_adv, new_pred = result
            score += 1
            dist = dist_func(x, x_adv)
            dists += dist
    return score/len(test_set), dists/len(test_set)

In [ ]:
evaluate(strategy22, dist_func, X_test)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# evaluate(strategy22, dist_func, X_test)